In [ ]:
import os
import numpy as np
import requests # permission
from bs4 import BeautifulSoup # webscraping package


In [ ]:
url = "https://developer.imdb.com/non-commercial-datasets/"
requests.get(url)

In [ ]:
url1 = "https://www.worldometers.info/coronavirus/"
requests.get(url1)

In [ ]:
url2 = "https://www.ambitionbox.com/"
requests.get(url2)

In [ ]:
url3 = "https://www.imdb.com/"
requests.get(url3)

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; X64) Apple WeKit /537.36(KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

In [ ]:
html = requests.get("https://www.imdb.com/",headers=headers).text


In [ ]:
soup = BeautifulSoup(html)
soup

In [ ]:
soup.find('title').getText()

In [ ]:
soup.findAll('h1')

In [ ]:
soup.findAll('h2',{'class':'ipc-title__text'})

In [ ]:
titlelist = soup.findAll('h3', {'class':'ipc-title__text'})
titlelist

In [ ]:
for title in titlelist:
    print(title.getText())

# https://www.worldmeters.info/coronavirus/#countries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
import sklearn, scipy
from datetime import date, datetime
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import gc

In [ ]:
url = "https://www.worldometers.info/coronavirus/#countries"
requests.get(url).text

In [ ]:
url = 'https://www.worldometers.info/coronavir/#countries'
requests.get(url).text

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; X64) Apple WeKit /537.36(KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

In [ ]:
html = requests.get("https://www.worldometers.info/coronavirus/#countries",headers=headers).text

In [ ]:
page_soup = BeautifulSoup(html, 'html.parser')
page_soup

In [ ]:
today = datetime.now()
print(today)

In [ ]:
yesterday = "%s %d, %d" %(date.today().strftime('%b'), today.day-1,today.year)
print(yesterday)
print()
print(today)

In [ ]:
date.today().strftime('%b')


In [ ]:
table = page_soup.findAll("table",{"id":"main_table_countries_yesterday"})
table

In [ ]:
containers = table[0].findAll('tr',{"style":""})
title = containers[0]
title

In [ ]:
containers[1]

In [ ]:
del containers[0]

In [ ]:
table = page_soup.findAll("table",{'id':'main_table_countries_yesterday'})
containers = table[0].findAll('tr',{"style":""})
title = containers[0]
del containers[0]

all_data = []
clean = True

for country in containers:
    country_data = []
    country_container = country.findAll('td')
    if country_container[1].text=='China':
        continue
    for i in range(1, len(country_container)):
        final_features = country_container[i].text
        if clean:
            if i != 1 and i!=len(country_container)-1:
                final_features = final_features.replace(",","")
                if final_features.find("+") != -1:
                    final_features = final_features.replace("+","")
                    final_features = float(final_features)
                #if final_features.find("_") != -1:
                    #final_features = float(final_feature)*-1
            if final_features == 'N/A':
                final_features = 0
            elif final_features =="" or final_features == " ":
                final_features = -1
            country_data.append(final_features)
    all_data.append(country_data)

In [ ]:
all_data

In [ ]:
df = pd.DataFrame(all_data)
df.head()

In [ ]:
df.drop([15,16,17,18,19,20],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
column_labels = ['Country','Total Cases','New Cases','Total Deaths','New Deaths','Total Recovered',
                 'New Recovered', 'Active Cases','Serious Critical','Total Cases/1M pop','Death/1M pop',
                 'Total Tests', 'Tests/1M pop', 'Population', 'Continent']

In [ ]:
df.columns= column_labels
df

In [ ]:
df.info()

In [ ]:
for label in df.columns:
    if label != 'Country' and label != 'Continent':
        df[label] = pd.to_numeric(df[label])

In [ ]:
df.info()

In [ ]:
sns.relplot(x='Total Cases',y='Total Recovered',data = df,kind='line')
plt.show()

In [ ]:
df["%inc Cases"] = df['New Cases']/df['Total Cases']*100
df["%inc Deaths"]= df['New Deaths']/df['Total Deaths']*100
df["%inc Recovered"]=df['New Recovered']/df['Total Recovered']*100

In [ ]:
df.head()

In [ ]:
cases = df[['Total Cases', 'Active Cases','Total Deaths']].loc[0]
cases_df =pd.DataFrame(cases).reset_index()
cases_df.columns=['Type','Total']
cases_df['Percentage']=np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df['Virus']=["COVID-19" for i in range(len(cases_df))]
fig = px.bar(cases_df, x= 'Virus',y= 'Percentage', color ='Type', hover_data=['Total'])
fig.show()

In [ ]:
cases = df[['New Cases','New Deaths', 'New Recovered']].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns=['Type','Total']
cases_df['Percentage']=np.round(100*cases_df['Total']/np.sum(cases_df['Total']),2)
cases_df['Virus']=["COVID-19" for i in range(len(cases_df))]
fig = px.bar(cases_df,x='Virus',y='Percentage',color='Type',hover_data=['Total'])
fig.show()

In [ ]:
df1 = df.drop([len(df)-1])
country_df = df1.drop([0])
country_df

In [ ]:
country_df.shape

In [ ]:
Top_Countries = 5
country = country_df.columns[1:14]
fig = go.Figure()
c =0 
for i in country_df.index:
    if c <Top_Countries:
        fig.add_trace(go.Bar(name=country_df['Country'][i],
                             x=country,y=country_df.loc[i][1:14]))
    else:
        break
    c +=1
fig.update_layout(title={"text":f'top {Top_Countries} countries affected'},yaxis_type="log")
fig.show()

In [ ]:
df['Continent'].value_counts()


In [ ]:
continent_df = df.groupby('Continent').sum().drop('All')
continent_df = continent_df.reset_index()
continent_df

In [ ]:
cases_list = ["Total Cases","Total Deaths","Total Recovered","New Cases", "New Deaths"]

In [ ]:
def continent_visualization(v_list):
    for label in v_list:
        c_df = continent_df[['Continent',label]]
        c_df['Percentage'] =np.round(100*c_df[label]/np.sum(c_df[label]),2)
        c_df['Virus']=["COVID-19" for i in range(len(c_df))]
        fig = px.bar(c_df, x= 'Virus', y= 'Percentage',color='Continent',hover_data =[label])
        fig.update_layout(title={"text":f"{label}"})
        fig.show()
        gc.collect()

In [ ]:
continent_visualization(cases_list)